# DU IA et Santé - Atelier NLP

Dans cet atelier, nous allons voir plusieurs techniques de NLP "moderne", principalement basées sur les transformers et la librairie [huggingface](https://huggingface.co/).

Cet atelier est diviser en trois partie.

Dans la partie 1, nous allons nous intéresser à la partie encodage des transformers (BERT et cie.) afin de mieux comprendre les représentations internes de ceux-ci.

Dans la partie 2, nous interesserons à la partie decodage des transformers (GPT et cie.) et aux méthodes de *prompt engineering* permettant d’améliorer leurs résultats.

Enfin, dans la partie 3, nous allons voir comment méler les deux approches pour construire des assistants personnels.

Mais, avant toute choses, il nous faut installer quelques librairies.

Pour cela, executez la cellule suivante:

In [ ]:
!pip install accelerate flash-attention jupyter-scatter

Puis, nous allons importer quelques librairies et vérifier que nous accédons bien au GPU (si vous en avez un).

Pour cela exécutez la cellule ci-dessous.

In [ ]:
# Librairies utilitaires
from tqdm.auto import tqdm
import gc
import re
import json

# Librairies mathématiques
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline

# Librairies ML
import torch
print("cuda available?", str(torch.cuda.is_available()))
device = torch.device("cpu")
if torch.cuda.is_available():
  device = torch.device("cuda:0")
  torch.cuda.set_device(device)
  print("cuda version:", torch.version.cuda)
  print("cuDNN enabled?", torch.backends.cudnn.enabled)
  print("cuDNN version:", torch.backends.cudnn.version())
  print("Device name? ", torch.cuda.get_device_name(torch.cuda.current_device()))

# Initialisation générateurs de nombres aléatoires
SEED = 42
np.random.seed(SEED)
torch.random.manual_seed(SEED)

Si le résultat est : `cuda available? False`, cela signifie que la librairie *torch* n’a pas pu accéder à votre GPU.

Dans le cas contraire (ou si vous n’avez pas de GPU à disposition), nous pouvons passer à la suite.

remarque: si vous utiliser ce GoogleCollab, cliquez sur Exécution > Modifier le type d’exécution et séléctionnnez l’option T4 GPU

## Partie 1 - Encodage et représentations internes

Dans cette partie, nous allons voir comment les transformers, plus spécifiquement les modèles de type BERT, encodent et transforment les textes fournis pour les donner en entrée des modèles. Nous allons aussi étudier les représentations internes de ce modèles, leurs espaces latents, et comment manipuler ces réprésentations.

Pour cela, nous allons nous baser sur le modèle [CamemBERT](https://camembert-model.fr/), entrainé sur des textes en français.

Si vous souhaitez essayer un autre modèle, voir: https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=trending

In [ ]:
from transformers import AutoModel, AutoTokenizer

bert_model_name = "Lajavaness/sentence-camembert-base"

bert_model = AutoModel.from_pretrained(bert_model_name, device_map=device, torch_dtype="auto")
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

### 1.1 Tokenization

La première étape pour permettre à un modèle de language de traiter du texte est la tokenisation.

Cette étape permet de découper une phrase ou un mot en un ou plusieurs *token* connus du modèle (l’ensemble des tokens connus par un modèle est appelé son "vocabulaire").

Par exemple:

In [ ]:
sentence = "fromage"
tokenized_sentence = bert_tokenizer.tokenize(sentence)
tokenized_sentence

Cependant, un modèle attend en entrée non pas une liste de tokens, mais la liste des id correspondant à ces tokens dans son vocabulaire.

Ainsi, il nous faut encoder les tokens obtenus précédemment en une liste d’entier utilisable par le modèle.

In [ ]:
encoded_sentence = bert_tokenizer.encode(tokenized_sentence, is_split_into_words=True, return_tensors="pt")
encoded_sentence

Le résultat ci-dessus présente la liste des ids correspondant aux tokens de notre phrase.

Nous pouvons cependant noter la précense de deux ids supplémentaires (5 et 6), ceux-ci correspondants aux tokens de début et fin de phrase.

Sentez-vous libre de modifier la phrase d’exemple avant de passer à la suite.

### 1.2 Word embeddings et espaces latents

Maintenant que nous avons comment tokeniser et encoder un texte pour permettre à un modèle de manipuler ce texte, nous allons voir les réprentations internes utilisé par ces modèles pour manipuler ces textes.

Pour cela nous allons utiliser la classe *pipeline* de la librairie *transformers*.

In [ ]:
from transformers import pipeline

Cette classe permet d’utiliser nos modèles sur divers problèmes.

Ici, nous allons l’utiliser pour effectuer de la *feature-extraction* et ainsi obtenir les représentations interne des phrases fournies à notre modèle.

In [ ]:
feature_extractor = pipeline('feature-extraction', model=bert_model, tokenizer=bert_tokenizer)

Par exemple:

In [ ]:
example_embeddings = feature_extractor(sentence)
pd.DataFrame(example_embeddings[0])

Nous voyons ici, que notre modèle transforme chaque token de notre mot en vecteurs, ou *embeddings*, de taille:

In [ ]:
len(example_embeddings[0][1])

Pour une meilleure manipulation, il est possible de faire la moyenne de ces vecteurs en un vecteur unique.

In [ ]:
example_embedding = pd.DataFrame([np.array(example_embeddings[0]).mean(axis=0)])
example_embedding

La taille de ces vecteurs determine le nombre de dimensions de l’espace latent de notre modèle.

Chaque dimension correspond à un "concept" utilisé par le modèle pour différencier les *tokens* entre eux.

Chaque vecteur correspond alors à un point dans cet espace, et chaque cellule d’un vecteur correspond à la position, allant de -1 à 1, du *token* associé sur
une des dimensions de l’espace latent du modèle.

Nous pouvons, entre autres, visualiser la position d’un *token* sur chaque dimension à l’aide d’une *heatmap*.


In [ ]:
# Représentation des 10 premières valeurs pour plus de lisibilité
sb.heatmap(example_embedding.iloc[:,:10], vmin=-1.0, vmax=1.0, annot=True, fmt=".2f")


Il est aussi possible d’avoir un aperçu du vocabulaire d’un modèle dans son espace latent.

Pour cela, il nous faut dans un premier temps recupérer les embeddings de chaque token présent dans le vocabulaire d’un modèle.

In [ ]:
# On récupère le vocabulaire du modèle
vocab = [token.replace("▁", "") for token in list(bert_tokenizer.get_vocab().keys())]

# On encodage le vocabulaire
embedded_vocab = feature_extractor(vocab, batch_size=32)

# On met le résultat sous un format plus facile à utiliser
df_embedded_vocab = pd.DataFrame([np.array(ev[0]).mean(axis=0) for ev in embedded_vocab], index=vocab)

# On affiche le résultat
df_embedded_vocab

On peut alors récupérer l’embedding d’un token specifique, par exemple:

In [ ]:
df_embedded_vocab.loc[[sentence]]

Il nous faut maintenant représenter ces tokens sur un plan en deux dimensions.

En d’autres termes, il nous faut réduire le nombre de dimensions de nos vecteurs
(ou *embeddings*) à 2.

Pour cela, nous allons utiliser l’algorithme *Principal Component Analysis* (PCA) disponible dans la librairie *scikit-learn*.

In [ ]:
from sklearn.decomposition import PCA

Dans un premier temps, nous allons initialiser PCA et l’appliquer aux vecteurs correspondants au vocabulaire de notre modèle pour déterminer les deux dimensions à utiliser.

In [ ]:
# On initialise PCA
pca = PCA(n_components=2)

# On applique PCA à nos embeddings
pca_embedded_vocab = pca.fit_transform(df_embedded_vocab)

# Et on mets le résultats sous un format plus facilement manipulable
df_pca = pd.DataFrame(data = pca_embedded_vocab, columns = ['PC1', 'PC2'], index=vocab)

Nous pouvons maintenant générer un graphe de points en deux dimensions dans lequel chaque point correspond à un token.

Pour cela nous utilisons les librairies *jscatter* et *ipywidgets* qui permettent de générer des graphes interactifs.

In [ ]:
import jscatter
import ipywidgets

In [ ]:
# Création du graphe
scatter = jscatter.Scatter(data=df_pca, x="PC1", y="PC2")

# Création du module interactif
output = ipywidgets.Output()

# Définition d'une fonction à appeler lors de la sélection de points
@output.capture(clear_output=True)
def selection_change_handler(change):
  display(df_pca.iloc[change.new])

# Connection de l'action de sélection de points à la fonction
scatter.widget.observe(selection_change_handler, names=["selection"])

# Affichage du graphe
ipywidgets.HBox([scatter.show(), output])

Pour d’autres exemples de visualisation d’espace latent, voir:

https://projector.tensorflow.org/

https://helboukkouri.github.io/embedding-visualization/

Chaque point d’un espace latent étant associé à un token et positionné les uns par rapport aux autres, il est donc théoriquement possible de calculer une "distance sémantique" entre deux tokens.

Pour cela, nous allons utiliser la [similarité cosinus](https://fr.wikipedia.org/wiki/Similarit%C3%A9_cosinus) disponible dans la librairie *scikit-learn*.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

La similarité cosinus se base sur l’angle formé par deux vecteurs de taille $n$ et fourni un score entre -1 et 1.

C’est une similarité généralement employée en TAL.

Nous allons donc l’employer ici pour trier le vocabulaire de notre modèle et trouver les tokens "proches" d’un embedding donné.

In [ ]:
### Définition de la fonction de tri
def sort_vocab_by_similarity(embedding):
  # On calcule la similarite entre l'embedding fourni en entree
  # et les embeddings du vocabulaire de notre modèle
  cos_sim = cosine_similarity(embedding, df_embedded_vocab)

  # On met les résultats sous un format plus facilement manipulable
  df = pd.DataFrame(cos_sim[0], columns=["similarity"], index=vocab)

  # On tri par similarite avec l'embedding fourni en entree
  df = df.sort_values(by=['similarity'], ascending=False)

  # On retourne le résultat
  return df

Par exemple:

In [ ]:
sort_vocab_by_similarity(df_embedded_vocab.loc[[sentence]])

C’est cette méthode qui est employé sur des sites tels que:

https://cemantix.certitudes.org/

https://degaucheoudedroite.delemazure.fr/

Enfin, il est aussi possible d’effectuer des opérations mathématiques sur nos vecteurs.

Nous avons déjà vu que nous pouvons faire une moyenne de plusieurs *embeddings*, voyons maintenant ce qu’il se passe lorsque nous additionnons et soustrayons des *embeddings* entre eux.

In [ ]:
mot1 = feature_extractor("France")
mot1 = pd.DataFrame([np.array(mot1[0]).mean(axis=0)])
mot1

In [ ]:
mot2 = feature_extractor("Camembert")
mot2 = pd.DataFrame([np.array(mot2[0]).mean(axis=0)])
mot2

In [ ]:
mot3 = feature_extractor("Mozzarella")
mot3 = pd.DataFrame([np.array(mot3[0]).mean(axis=0)])
mot3

In [ ]:
mot1 - mot2 + mot3

In [ ]:
sort_vocab_by_similarity(mot1 - mot2 + mot3)[:10]

C’est ce principe d’addition de "concept" qui est utilisé, par exemple, dans: https://neal.fun/infinite-craft/

Pour d’autres exemples employant d’autres modèles, voir: http://nlp.polytechnique.fr/word2vec

Avant de passer à la suite, nous allons libérer des éléments de la mémoire de notre session.

In [ ]:
del embedded_vocab
del df_embedded_vocab
del vocab
del scatter
gc.collect()

Et pour en savoir plus sur le fonctionnement des embeddings, voir: https://jalammar.github.io/illustrated-word2vec/

### 1.3 Sauvegarde et recherche sémantique de documents

Dans la section précédente, nous avons vu comment transformer des mots sous forme de vecteurs et comment manipuler ces vecteurs.

Nous allons maintenant voir comment utiliser ces méthodes pour manipuler des phrases et des textes plus complexes.

De la même manière qu’un mot, une phrase va être découpé en *tokens* et chaque *token* transformé en *embedding*.

In [ ]:
pd.DataFrame(feature_extractor("J'aime le fromage")[0])

Bien qu’il existe plusieurs approches possibles, la méthode généralement pour transformer une phrase, un texte ou un document en un vecteur est simplement de faire la moyenne des *embeddings*.

Par exemple:

In [ ]:
example1 = pd.DataFrame([feature_extractor("J'aime le fromage",return_tensors = "pt")[0].numpy().mean(axis=0)])
example1

In [ ]:
example2 = pd.DataFrame([feature_extractor("Je déteste le fromage",return_tensors = "pt")[0].numpy().mean(axis=0)])
example2

In [ ]:
example3 = pd.DataFrame([feature_extractor("Le fromage, c'est vraiment bon", return_tensors = "pt")[0].numpy().mean(axis=0)])
example3

In [ ]:
example4 = pd.DataFrame([feature_extractor("Le soleil brille sur la ville de Lyon aujourd'hui", return_tensors = "pt")[0].numpy().mean(axis=0)])
example4

Et de la même manière que pour les mots, il est possible de calculer une "distance sémantique" entre différents textes.

In [ ]:
cosine_similarity(example1, example2)

In [ ]:
cosine_similarity(example1, example3)

In [ ]:
cosine_similarity(example1, example4)

En se basant sur ce principe, il est alors possible de stocker des textes et leurs *embeddings* afin de les retrouver en fonction de leur "similarité" avec une requête données.

Pour cela, il nous faut d’abord créer une fonction permettant de stocker des textes et leurs *embeddings*.

In [ ]:
# On cree une liste d'embeddings vide
bdd_embeddings = pd.DataFrame(columns=example1.columns)

# On fait de meme pour stocker les textes
bdd_texts = []

def store_text_embeddings(text):
  # Au cas ou, on test si le texte n'existe pas deja
  if not text in bdd_texts:
    # On genere les embeddings
    embeddings = feature_extractor(text, return_tensors = "pt")[0].numpy().mean(axis=0)
    # On ajoute les embeddings a notre liste d'embeddings
    bdd_embeddings.loc[len(bdd_embeddings.index)] = embeddings
    # On ajoute notre texte a notre liste de textes
    bdd_texts.append(text)

On peut maintenant utiliser cette fonction pour stocker en mémoire différents textes.

Par exemple:

In [ ]:
texts = [
"Camembert est une appellation générique qui désigne généralement un fromage à pâte molle et à croûte fleurie. Commercialement, cette appellation d'origine normande ne fait l'objet d'aucune protection et se voit utilisée pour des fromages n'ayant parfois que peu de rapport avec le camembert originel. Dans certaines régions de France, le camembert est appelé « claquos », « clacos », « calendos ».",
"Le bleu d’Auvergne est un fromage à pâte persillée fabriqué en France dans le Massif central à partir de lait de vache. Son persillage allant du bleu au bleu noir. Son appellation d'origine bénéficie de protections depuis 1975.",
"La tomme de Savoie est un fromage produit en France dans la région alpine de Savoie, regroupant les départements de la Savoie et de la Haute-Savoie. Son appellation est protégée par une indication géographique protégée.",
"Le munster ou munster-géromé (ou encore Minschterkäs en francique lorrain ou Minschterkaas en alsacien) est un fromage à pâte molle fabriqué à partir de lait de vache dans l'Est de la France. Son appellation est protégée nationalement depuis 1969 par une appellation d'origine contrôlée (AOC) et dans l'ensemble des pays de l'Union européenne depuis 1996 par une appellation d'origine protégée (AOP).",
"Les bries sont une famille de fromages à pâte molle à croûte fleurie, originaire de la région française de Brie.",
"L’emmentaler ou emmental est un fromage d'origine suisse à pâte dure dont le nom provient de la vallée de l'Emme (en allemand, Emmental), une région à l'est du canton de Berne.",
"Beaufort (/bofɔːʁ/) est l'appellation d'origine d'un fromage au lait cru de vache, à pâte pressée cuite, élaboré en Savoie en France. La production du lait et sa transformation s'effectuent dans une aire comprenant la région du Beaufortain d'où il tire son nom. Il est formé en meule à talon légèrement concave. L'appellation beaufort est préservée commercialement via une Appellation d'origine protégée.",
"Le gorgonzola est l'appellation d'origine d'un fromage traditionnel à base de lait de vache, à pâte persillée, fabriqué dans les régions de Lombardie et du Piémont.",
"Le Valençay est une appellation d'origine désignant un fromage de chèvre au lait cru du Berry, et de la région Centre-Val de Loire, en France. Elle reprend le nom de la commune homonyme.",
"Le saint-marcellin IGP est un fromage français du Dauphiné. Son Indication géographique protégée (IGP) date de la fin 2013, elle s'étend sur 274 communes en Isère, dans la Drôme et en Savoie.",
"Reblochon ou reblochon de Savoie est une appellation d'origine désignant un fromage français produit principalement en Haute-Savoie et dans quelques communes de Savoie. Cette appellation est originaire du massif des Bornes et des Aravis, principalement la vallée de Thônes, et s'est étendue au val d'Arly et au massif des Bauges.",
"Comté est l'appellation d'origine d'un fromage français transformé principalement en Franche-Comté et bénéficiant d'une AOC depuis 1958 et d'une AOP depuis 1996. Son aire de production s'étend dans les départements du Jura, du Doubs, et de l'est de l'Ain. Elle englobe également une commune de Haute-Savoie et quelques-unes de Saône-et-Loire."
]

for text in texts:
  store_text_embeddings(text)

Si on affiche nos listes de textes et d’*embeddings*, on voit que les textes et leurs *embeddings* de la cellule précédente ont bien été enregistrés.

In [ ]:
bdd_texts

In [ ]:
bdd_embeddings

Ensuite, il nous faut créer une fonction permettant de rechercher dans ces *embeddings* pour trouver les documents stockés les plus "similaires" à une requête.

In [ ]:
def find_embedded_texts(query, nresults=5):
  # On genere l'embedding correspondant a la requete
  query_embeddings = pd.DataFrame([feature_extractor(query, return_tensors = "pt")[0].numpy().mean(axis=0)])

  # On calcule la distance entre l'embedding de la requete et les embeddings stockes dans notre liste
  df = pd.DataFrame(cosine_similarity(query_embeddings, bdd_embeddings)[0], columns=["similarity"])

  # On tri par similarite
  df = df.sort_values(by=['similarity'], ascending=False)

  # On recupere les indices des premiers resultats
  indexes = df.iloc[:nresults].index

  # On recupere les textes correspondants
  results = []
  for i in indexes:
    results.append({"doc": bdd_texts[i], "score": df.loc[i]["similarity"]})

  # On retourne les textes et leurs scores de similarite
  return results

À l’aide de cette fonction, nous pouvons maintenant rechercher parmis les textes stockées ceux qui correspondent à une requête.

Par exemple:

In [ ]:
query = """
Fromage de Savoie
"""

find_embedded_texts(query)

Ici, pour un objectif pédagogique, nous avons codé nous même le fonctionnement d’une base de données utilisant les principe des *embeddings*.

Il existe cependant des librairies python, telle que [chroma](https://www.trychroma.com/), qui permettent de gérer plus efficacement de telles bases de données.

## Partie 2 - Large modèles de langage et génération de texte

Dans cette partie, nous allons aborder l’aspect "decodage" des transformers.

Cet aspect est généralement employé pour générer du texte et est la base des large modèles de langage (LLM) tels que GPT-4, Mistral, ou encore LLaMA.

Pour cet atelier, nous allons utiliser le modèle [Phi-3](https://azure.microsoft.com/en-us/blog/introducing-phi-3-redefining-whats-possible-with-slms/). Un modèle proposé par Microsoft, qui a l’avantage d’être assez léger et sous licence libre (MIT).

Afin de charger ce modèles, nous allons utiliser la classe *AutoModelForCausalLM* de la librairie *transformers* comme suit.

In [ ]:
from transformers import AutoModelForCausalLM

In [ ]:
llm_model_name = "microsoft/Phi-3-mini-128k-instruct"

llm_model = AutoModelForCausalLM.from_pretrained(llm_model_name, device_map=device, torch_dtype="auto", trust_remote_code=True)
llm_tokenizer = AutoTokenizer.from_pretrained(llm_model_name)


Si vous souhaitez utiliser un autre modèle, voir: https://huggingface.co/models?pipeline_tag=text-generation.

### 2.1 Complétion de texte

L’objectif de base d’un modèle de langage est la complétion de texte.

En d’autres termes, pour un texte données, un llm va chercher à déterminer le *token* pouvant le plus plausiblement faire suite à ce texte.

Pour ce faire, nous allons créer un nouveau pipeline permettant de gérérer du texte à partir de notre llm.

In [ ]:
text_generator = pipeline(
    "text-generation",
    model=llm_model,
    tokenizer=llm_tokenizer,
)

Nous alors lui demander de compléter un texte.

Par exemple:

In [ ]:
prompt = "Le fromage c'est vraiment bon, surtout le"

Pour ce faire, notre llm va décomposer la phrase en *token* puis en *embeddings* comme vu en Partie 1.

Puis il va déterminer, parmis les *tokens* de son vocabulaire, lesquels sont les plus "plausible" pour faire suite aux tokens composant le texte fourni en entrée.

In [ ]:
output = text_generator(prompt, max_new_tokens = 1, return_full_text = False, do_sample = True, num_return_sequences=5)
print(output)

Parmis ces *tokens*, notre llm va prend le plus "plausible", puis l’ajouter aux texte fournit en entrée pour déterminer le *token* suivant.

Ainsi de suite, jusqu’à atteindre la longueur souhaitée.

In [ ]:
output = text_generator(prompt, max_new_tokens = 5, return_full_text = False)
print(output[0]["generated_text"])

Il est aussi possible de pousser le llm à sélectioner des *tokens* qui seraient moins "plausible", de manière plus aléatoire.

Pour cela, il suffit de lui fournir une "temperature". Plus cette "temperature" est élevée, plus le llm cherchera des *tokens* moins plausible.

In [ ]:
output = text_generator(prompt, max_new_tokens= 5, return_full_text = False, do_sample=True, temperature = float(SEED) / 50.0, num_return_sequences=3)
print(output)

### 2.2 Agents Conversationnels (ChatBot)

La complétion de texte que nous avons vu précédemment est la base des agents conversationnels, ou ChatBot, tels que [ChatGPT](https://chatgpt.com/), [Copilot](https://www.bing.com/chat?q=Microsoft+Copilot&FORM=hpcodx) ou encore [ChatCGT](https://chatcgt.fr/).

Néanmoins, afin d’optimiser cette complétion dans le cadre d’un ChatBot, les modèles employés sont entrainé sur des textes tournés sous forme de conversations ou "instructions".

C’est le cas du modèle que nous utilisons. D’ailleurs, si vous augmentez la taille des textes générés dans l’example précédent, notre llm devrait générer une conversation.

Afin d’utiliser notre llm dans un ChatBot, il nous donc saisir notre requête en précisant que c’est le texte saisie par l’utilisateur.

Par exemple:

In [ ]:
query = "J'aimerais manger un fromage de Savoie, que me conseille tu ?"

messages = [
    {"role": "user", "content": query},
]

In [ ]:
output = text_generator(messages, return_full_text = False, max_new_tokens = 500)
print(output[0]['generated_text'])

Cependant, notre llm ne garde pas en mémoire le contexte de la conversation et les questions déjà posées.

Pour poursuivre cette conversation, il nous faut donc ajouter les réponses du ChatBot aux messages pré-existant avant de poser une nouvelle question.

Par exemple:

In [ ]:
messages.append({"role": "assistant", "content": output[0]['generated_text']})
messages.append({"role": "user", "content": "Je cherche un fromage plus insolite"})
messages

In [ ]:
output = text_generator(messages, return_full_text = False, max_new_tokens = 500)
print(output[0]['generated_text'])

À partir de là, nous avons les éléments pour créer un ChatBot minimal.

Tout d’abord, nous allons créer un message qui va initialiser le ChatBot avec ce qui est attendu de lui. Ce message initial est aussi appelé *pre-prompt*.


In [ ]:
pre_prompt = """
Tu es un ChatBot spécialisé en fromages.
L'utilisateur est un amateur de fromages de longue date et il connait lui aussi de nombreux fromages.
Ton objectif est de lui faire découvrir des fromages peu connu et insolites en fonction de ses demandes.
Acceuille le avec un message de bienvenu et en te presentant.
"""

messages = [
    {
        "role": "user",
        "content": pre_prompt
    }
]

Ensuite, nous allons définir les paramètres généraux à utiliser lors de la génération de textes par le ChatBot.

In [ ]:
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "do_sample": False,
    # pour utiliser le paramètre "temperature", passer le paramètre "do_sample" à True
    #"temperature": float(SEED)
}

Et enfin nous pouvons lancer une boucle qui génére un message du ChatBot, attends un message de l’utilisateur et complète automatiquement la conversation.

La conversation s’arrête lorsque l’utilisateur saisi le mot "fin".

In [ ]:
usr_input = ""
while usr_input != "fin":
  # On genere un message du ChatBot
  output = text_generator(messages, **generation_args)

  # On affiche le message
  print("\nChatBot:", output[0]['generated_text'], "\n")

  # On attend l'entree de l'utilisateur
  usr_input = input("Utilisateur:")

  # On met à jour la conversation
  messages.append({"role": "assistant", "content": output[0]['generated_text']})
  messages.append({"role": "user", "content": usr_input})


### 2.3 Techniques d’instructions (Prompt Engineering)

Avant de clôre cette partie, nous allons aborder différentes techniques, issu du domaine du *Prompt Engineering*, permettant d’optimiser et maximiser la qualité des réponses fournie par un agent conversationnel.

Parmis ces techniques, une très simple a employé est celle de la chaîne de pensée, ou *Chain of Thoughts* (CoT) en anglais.

Pour utiliser cette technique, il suffit d’ajouter "Pensons étape par étape" (*Let's think step by step*) à notre requête.

Cela forcera la probabilité que le ChatBot génère un texte avec plusieurs étapes, donc un contenu plus détaillé et donc une réponse de meilleure qualité.

Pour en savoir plus, voir: https://www.promptingguide.ai/techniques/cot

In [ ]:
messages = [
    {
        "role": "user",
        "content":
        """
          Pensons étape par étape.
        """ + query
     }
]

In [ ]:
output = text_generator(messages, **generation_args)
print(output[0]['generated_text'])

Cependant, la plupart des llm sont maintenant entrainé pour utiliser automatiquement du CoT lors de la génération de leurs réponses.

Une autre technique possible, dérivée de CoT, est l’arbre de pensées, ou *Tree of Thoughts* (ToT) en anglais.

Cette technique reprends le principe de CoT, mais détaillant étape par étape des réponses selon différents point de vue et en comparant ces réponses entre elles.

Pour en savoir plus, voir: https://www.promptingguide.ai/techniques/tot

In [ ]:
messages = [
    {
        "role": "user",
        "content":
        """
          Imagine que trois experts différents débattent pour répondre à cette question.
          Chaque expert écrit une étape de sa réflexion et la partage avec le groupe.
          Puis tous les experts continue ainsi de suite, étape par étape.
          Si un expert réalise qu'il s'est trompé à un moment de sa réflexion, il sort de la discussion.
          La question est:
        """ + query
     }
]

In [ ]:
generation_args["max_new_tokens"] = 5000

In [ ]:
output = text_generator(messages, **generation_args)
print(output[0]['generated_text'])

Enfin, il est aussi possible de demander au ChatBot de générer des connaissances générales en lien avec la question posée.

Cette technique, appellée *Generated Knowledge Prompting*, guide alors la génération de texte à l’aide de ces connaissances fourni explicitement.

Pour en savoir plus, voir: https://www.promptingguide.ai/techniques/knowledge

In [ ]:
messages = [
    {
        "role": "user",
        "content":
        """
          Avant de répondre à la question, écrit les différentes connaissances que tu as sur le sujet.
          La question est:
        """ + query
     }
]

In [ ]:
output = text_generator(messages, **generation_args)
print(output[0]['generated_text'])

Bien entendu, ces différentes techniques ne sont pas mutuellement exclusives et pouvent se combiner entre elles.

Par exemple:

In [ ]:
messages = [
    {
        "role": "user",
        "content":
        """
          Imagine que trois experts différents débattent pour répondre à cette question.
          Chaque expert a des connaissances différentes des autres sur le sujet.
          Avant de débattre, chaque expert énonce ses connaissances sur le sujet.
          Ensuite, chaque expert écrit une étape de sa réflexion et la partage avec le groupe.
          Chaque expert peut réfuter les arguments d'un autre expert.
          Si un expert réalise qu'il s'est trompé à un moment de sa réflexion, il sort de la discussion.
          Ainsi de suite jusqu'à arriver à un consensus apportant une réponse à la question.
          La question est: "Je cherche un fromage de Savoie, que me conseille-tu?"
        """
     }
]

In [ ]:
output = text_generator(messages, **generation_args)
print(output[0]['generated_text'])

De nombreuses autres techniques de *Prompt Engineering* existent, telles que [Self-Consistency](https://www.promptingguide.ai/techniques/consistency) ou [ReAct](https://www.promptingguide.ai/techniques/react), et bien d’autres vont surement voir le jour dans les prochaines années.

L’objectif étant alors de trouver les bonnes formulations pour répondre le plus efficacement à telle ou telle demande. Des techniques comme l’[APE](https://www.promptingguide.ai/techniques/ape) pour *Automatic Prompt Engineer* propose même de générer ces formulations via un modèle de langage.

Pour aller plus loin sur le sujet, voir: https://www.promptingguide.ai/

## Partie 3 - Combiner encodage et décodage avec les LLM Agents

Pour conclure cet atelier, nous allons voir dans cette partie comment créer un assistant personnel minimal en combinant les différentes techniques et méthodes vues dans les partie précédentes.

### 3.1 Generation augmentée par requêtes (RAG)

La géneration augmentée par requêtes, ou *Retrieval Augmented Generation* (RAG) en anglais, est une technique de *Prompt Engineering* consistant à compléter une question avec des informations vérifiées et stockées en base.

Nous pouvons, par exemple, réutiliser la fonction créée en partie 1 afin de retrouver des documents correspondants à notre requête.

In [ ]:
docs = find_embedded_texts(query)
docs

Ensuite, il nous faut créer une nouvelle requête complétant la première avec les informations retrouvées et indiquant à notre llm d’utiliser ses informations.

In [ ]:
usr_input = "Pour répondre à la question base toi sur les informations suivantes, en indiquant les informations utilisées et comment:"

for i, doc in enumerate(docs):
  usr_input += "\n\nInformation " + str(i) + ": \"" + doc["doc"] + "\""

usr_input += "\n\nla question est: \"" + query + "\""

print(usr_input)

Nous pouvons alors fournir cette nouvelle requête à notre llm, qui devrait générer une réponse basée sur les informations que nous avons retrouvées en base.

In [ ]:
messages = [
    {
        "role": "user",
        "content": usr_input
     }
]

output = text_generator(messages, return_full_text = False, max_new_tokens=500)
print(output[0]['generated_text'])

Pour en savoir plus sur le RAG, voir: https://www.promptingguide.ai/techniques/rag

### 3.2 Utilisation d’outils

Il est aussi possible d’indiquer au llm les fonctions, aussi appellé outils ou *tools*, qu’il peut utiliser pour répondre aux requêtes d’un utilisateur, cela afin d’automatiser leur utilisation.

Pour cela, il nous faut lui fournir un *prompt* avec les instructions nécessaires.

Par exemple:

In [ ]:
messages = [
    {
        "role": "user",
        "content":
          """
            Pour répondre à la question, utilise la fonction "find_embedded_texts" et base toi sur ses résultats en indiquant quelle information tu as utilisé et comment.

            Pour utiliser cette fonction, écrit uniquement et simplement son nom et ses arguments, comme suit:

            {"name": "find_embedded_texts", "args": {"query": "question de l'utilisateur"}}

            La question est:
          """ + query
    }
]

Ce *prompt* permet d’aiguiller la génération de texte afin que le llm écrive la fonction à appeler.

In [ ]:
generated_text = text_generator(messages, return_full_text = False, max_new_tokens=500)[0]['generated_text']
print(generated_text)

Il nous faut alors récupérer les éléments pour appeler la fonction.

In [ ]:
function_called = json.loads(re.findall(r'\{\"name\": \"find_embedded_texts\", \"args\": \{.+\}\}', generated_text)[0])
function_called

Pour ensuite appeler la fonction avec les arguments générés par le llm.

In [ ]:
docs = find_embedded_texts(function_called["args"]["query"])
docs

Puis compléter la discussion avec les résultats de la recherche.

In [ ]:
system_text = "Résultats de la fonction:"

for i, doc in enumerate(docs):
  system_text += "\n\nInformation " + str(i) + ": \"" + doc["doc"] + "\""

messages.append({"role": "assistant", "content": generated_text})
messages.append({"role": "user", "content": system_text})

messages

Pour enfin demander au llm de nous fournir une réponse basées sur les résultats retrouvé.

In [ ]:
generated_text = text_generator(messages, return_full_text = False, max_new_tokens=500)[0]['generated_text']
print(generated_text)

Dans l’exemple précédent, nous avons demandé au llm de rechercher des informations correspondants à une requête.

Mais, de manière similaire, il est tout à fait possible de lui demander d’enregistrer de nouvelles informations.

Par exemple:

In [ ]:
messages = [
    {
        "role": "user",
        "content":
          """
            Pour enregistrer des informations, utilise la fonction "store_text_embeddings".

            Pour utiliser cette fonction, écrit simplement son nom et ses arguments, comme suit:

            {"name": "store_text_embeddings", "args": {"text": "information à enregistrer"}}

            Enregistre l'information suivante: "La mozzarella ou mozzarelle est un fromage à pâte filée traditionnel de la cuisine italienne, à base de lait de vache ou de bufflonne. Deux de ces fromages bénéficient d'une appellation d'origine protégée, la mozzarella di bufala campana produite en Campanie avec du lait de bufflonne, et la mozzarella di Gioia del Colle produite dans les Pouilles avec du lait de vache"
          """
    }
]

In [ ]:
generated_text = text_generator(messages, return_full_text = False, max_new_tokens=500)[0]['generated_text']
print(generated_text)

In [ ]:
function_called = json.loads(re.findall(r'\{\"name\": \"store_text_embeddings\", \"args\": \{.+\}\}', generated_text)[0])
store_text_embeddings(function_called["args"]["text"])

In [ ]:
bdd_texts

In [ ]:
bdd_embeddings

Nous pouvons demander à notre llm de générer des fiches thématiques à partir d’une information données et de faire les appels correspondants.

Par exemple:

In [ ]:
messages = [
    {
        "role": "user",
        "content":
          """
            Pour enregistrer des informations, utilise la fonction "store_text_embeddings".

            Pour utiliser cette fonction, écrit simplement son nom et ses arguments, comme suit:

            {"name": "store_text_embeddings", "args": {"text": "information à enregistrer"}}

            Génère et enregistre 5 fiches, et seulement 5, de l'information suivante, chaque fiche portant sur un élément particulier:

            "Le roquefort ([ʁɔk(ə)fɔːʁ]), ou ròcafòrt en occitan rouergat, est un fromage à pâte persillée élaboré dans le sud de la France exclusivement avec des laits crus de brebis.
            La meilleure période de consommation de ce fromage français s'étend de janvier à août.
            Ce fromage est mentionné expressément pour la première fois au xie siècle2, ce qui en fait un symbole historique de la région des causses et vallées de l'Aveyron. Cette région rurale, établie sur un terroir parfois très difficile à exploiter, en a fait sa richesse financière et culturelle.
            De réputation internationale, il est associé à l'excellence de l'agriculture française et à sa gastronomie. Il est même devenu l'emblème de la résistance des producteurs et transformateurs de fromage au lait cru contre les demandes réitérées de la généralisation de la pasteurisation du lait[réf. souhaitée]. Il existe aujourd'hui sous forme industrielle et laitière, sa forme fermière subsiste toutefois encore mais marginalement.
            En 1925, en France, « roquefort » est la première appellation d'origine dont l'emploi, du moins dans sa désignation d'un fromage voué au commerce, est encadré administrativement.
            Depuis 1979, cette appellation bénéficie d'une appellation d'origine contrôlée (AOC) et, depuis 1996, d'une appellation d'origine protégée (AOP)3. Cette AOP se caractérise par les grottes afférentes aux fleurines, des failles dans la roche créant un flux d'air continu et donnant une humidité de 80 % minimum et une température de 10 °C en moyenne. Produit par sept fabricants4 sur une zone de 2 000 × 300 m, le fromage s'affine. Ces conditions permettent le développement du Penicillium roqueforti qui donne au fromage son goût unique.
            La marque Société du groupe Lactalis produit 70% du roquefort en France et domine donc largement le secteur de production de ce fromage5."
          """
    }
]

In [ ]:
generated_text = text_generator(messages, return_full_text = False, max_new_tokens=500)[0]['generated_text']
print(generated_text)

In [ ]:
find_calls = re.findall(r'\{\"name\": \"store_text_embeddings\", \"args\": \{.+\}\}', generated_text)
find_calls

In [ ]:
for call in find_calls:
  function_called = json.loads(call)
  store_text_embeddings(function_called["args"]["text"])

In [ ]:
bdd_texts

In [ ]:
bdd_embeddings

Pour plus d’information sur l’utilisation d’outils par les llm, voir: https://python.langchain.com/docs/use_cases/tool_use/

### 3.3 Création d’un agent


Nous avons maintenant tous les éléments pour créer notre propre agents LLM.

Tout d’abord, nous allons créer un *pre-prompt* pour l’initialiser avec les instructions necessaires à son fonctionnement.

In [ ]:
pre_prompt = """
Tu es un ChatBot spécialisé en fromages.

Tu as à ta disposition les fonctions suivantes:

[
  {
    "name": "find_embedded_texts",
    "args": [{
      "name": "query",
      "type": str
      }],
    "descr": "recherche en base de données des documents en lien avec la demande de l'utilisateur."
  },
  {
    "name": "store_text_embeddings",
    "args": [{
      "name": "text",
      "type": str
      }],
    "descr": "enregistre un texte en base de données pour pouvoir le rechercher plus tard"
  }
]

Pour utiliser une de ces fonctions, écrit simplement son nom et ses arguments, comme suit:

{"name": "store_text_embeddings", "args": {"text": "information à enregistrer"}}

{"name": "find_embedded_texts", "args": {"query": "question de l'utilisateur"}}

C'est à toi de le faire, pas à l'utilisateur.

Ne mentionne ces fonctions que pour les utiliser, pas pour les expliquer à l'utilisateur.

L'utilisateur est un amateur de fromages de longue date et il connait lui aussi de nombreux fromages.

Ton objectif est de lui faire découvrir des fromages peu connu et insolites en fonction de ses demandes.

Réponds à ses demandes en utilisant les fonctions à ta disposition, en détaillant tes réponses étape par étape et en indiquant quelles informations tu as utilisé et comment.

Acceuille le avec un message de bienvenu et en te presentant.
"""

Il nous faut aussi créer une fonction qui va nous permettre de détecter les fonctions à appeler.

In [ ]:
def call_functions(generated_text):
  find_calls = re.findall(r'\{\"name\": \".+\", \"args\": \{.+\}\}', generated_text)

  results = ""
  for call in find_calls:
    function_called = json.loads(call)
    if function_called["name"] == "store_text_embeddings":
      store_text_embeddings(function_called["args"]["text"])
      results += "\nRésultat de la fonction: document enregistré\n"
    elif function_called["name"] == "find_embedded_texts":
      docs = find_embedded_texts(function_called["args"]["query"])
      results += "\nRésultat de la fonction: " + str(docs) + "\n"
  return results

Et enfin, nous pouvons lancer notre agent LLM.

In [ ]:
# Initialisation du pre-prompt
messages = [
    {
        "role": "user",
        "content": pre_prompt
    }
]

# Initialisation des paramètres
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "do_sample": False,
    # pour utiliser le paramètre "temperature", passer le paramètre "do_sample" à True
    #"temperature": float(SEED)
}

# Lancement de l'agent
usr_input = ""
while usr_input != "fin":
  # On recupere le texte genere par l'agent
  output = text_generator(messages, **generation_args)
  messages.append({"role": "assistant", "content": output[0]['generated_text']})

  # On affiche le résultat
  print("\nChatBot:", output[0]['generated_text'], "\n")

  # On appelle les fonctions si besoin
  usr_input = call_functions(output[0]["generated_text"])

  # Si aucune fonction n'a ete appele on recupere l'entree de l'utilisateur
  if usr_input != "":
    print("\nSystem:", usr_input, "\n")
  else:
    usr_input = input("Utilisateur:")

  messages.append({"role": "user", "content": usr_input})


Pour des raisons pédagogiques, nous avons coder nous même les fonctionnalités de cette agent. Il existe cependant des librairies telles que [LangChain](https://python.langchain.com/v0.1/docs/get_started/introduction/) qui vous permetteront de développer plus efficacement un agent LLM.

Vous connaissez maintenant les principes de base vous permettant de construire un assistant personnel basé sur un LLM.

N’hésitez pas à modifier ce notebook selon vos envies et vos besoins 😀

Et si ce notebook vous a été utile, n’oubliez pas qu’il est sous licence [BeerWare](https://fr.wikipedia.org/wiki/Beerware) 😏